This has been optimized
- Add etf sharpes too

In [19]:
import time
import os 

start_time = time.time()

WORKSPACE_DIR = os.getenv('WORKSPACE_DIR')
if not os.getcwd().endswith('portfolio_py'):
    os.chdir(f'{WORKSPACE_DIR}/portfolio_py')
print(os.getcwd())

from utils.finance_utils import calculate_sharpe_ratio

/Users/blakeuribe/Desktop/portfolio_py


In [20]:
from dotenv import load_dotenv
import os

import yfinance as yf
import numpy as np
import pandas as pd

from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm


In [21]:
# Load environment variables
load_dotenv()
TODAY = datetime.today()
ONE_YEAR_AGO = TODAY - relativedelta(years=1)

In [22]:
# Download and import data
df = pd.read_json(f'{WORKSPACE_DIR}/portfolio_py/data/raw/company_tickers.json')
df = df.T.drop(columns=['cik_str', 'title'])

tbill_data = yf.download('^IRX', start=ONE_YEAR_AGO, end=TODAY, auto_adjust=True)['Close']
tbill_data = tbill_data / 100 / 360  # Convert to daily rate

[*********************100%***********************]  1 of 1 completed


In [23]:
# Initialize results list to store data
n_stocks = 7000
tickers = df['ticker'][0:n_stocks]


# Calculate Sharpe ratios for all tickers in one go
try:
    sharpe_ratios = calculate_sharpe_ratio(tickers, tbill=tbill_data, start_date=ONE_YEAR_AGO, end_date=TODAY)
    results_df = pd.DataFrame({
        'ticker': tickers,
        'sharpe_ratio': sharpe_ratios.values

    })
except Exception as e:
    print(f'Error calculating Sharpe ratios: {str(e)}')


# Check percentage of NaN values
print('\n')
print(f'Pct of NaN values is: {(results_df.isna().sum().sum() / len(results_df)) * 100}%')

# Save the results to CSV
results_df.to_csv(f'{WORKSPACE_DIR}/portfolio_py/data/clean/sharpe_ratios.csv', index=False)

# %%
end_time = time.time()
elapsed_time = end_time - start_time

print(f'Processing time: {elapsed_time:.2f} seconds, for {n_stocks}')



[                       1%                       ]  99 of 7000 completed

Error calculating Sharpe ratios: can't start new thread


Pct of NaN values is: 100.0%
Processing time: 3.22 seconds, for 7000


[                       1%                       ]  101 of 7000 completed

[***                    7%                       ]  456 of 7000 completedFailed to get ticker 'KITT' reason: Expecting value: line 1 column 1 (char 0)
[***                    7%                       ]  459 of 7000 completedFailed to get ticker 'LAES' reason: Expecting value: line 1 column 1 (char 0)
Failed to get ticker 'VLCN' reason: Expecting value: line 1 column 1 (char 0)
[***                    7%                       ]  464 of 7000 completedFailed to get ticker 'LIFD' reason: Expecting value: line 1 column 1 (char 0)
[***                    7%                       ]  465 of 7000 completedFailed to get ticker 'CBDY' reason: Expecting value: line 1 column 1 (char 0)
[***                    7%                       ]  468 of 7000 completedFailed to get ticker 'UGRO' reason: Expecting value: line 1 column 1 (char 0)
[***                    7%                       ]  471 of 7000 completedFailed to get ticker 'LQMT' reason: Expecting value: line 1 column 1 (char 0)
[***            